In [1]:
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import operator
import pickle

import src.fusiontools as fusiontools
import src.scoring as scoring
import src.postprocess as postprocess

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
path_to_results = 'results/'

# Load best models

In [41]:
# Load best recency model
recency_rank_dic_path = 'ranks_test/recency_beta_7000_rec-all'
with open(recency_rank_dic_path, 'rb') as infile:
    recency_ranks = pickle.load(infile)
    
# Load best knn model
knn_rank_dic_path = 'ranks_test/knn-k-50-rec-all'
with open(knn_rank_dic_path, 'rb') as infile:
    knn_ranks = pickle.load(infile)

# Load best bayesian model
bayes_rank_dic_path = 'ranks_test/bayesian_results_0-35785'
with open(bayes_rank_dic_path, 'rb') as infile:
    bayes_ranks = pickle.load(infile)


# Discard tail of recipients

In [34]:
nb_kept_recips = 100
knn_ranks_cropped = fusiontools.keep_only_max_recips(knn_ranks, nb_kept_recips)
recency_ranks_cropped = fusiontools.keep_only_max_recips(recency_ranks, nb_kept_recips)
bayes_ranks_cropped = fusiontools.keep_only_max_recips(bayes_ranks, nb_kept_recips)

# Prepare models and weights

In [35]:
models = [bayes_ranks_cropped, recency_ranks_cropped, knn_ranks_cropped]
weights = [0.35785, 0.39316, 0.20716]

#Fast check that dictionnaries are the right lenght
for model in models:
    print(len(model))

2362
2362
2362


In [28]:
# for mid, recipients in bayes_ranks_cropped.items():
#     print(len(recipients))


In [42]:
# models = [knn_ranks, recency_ranks]
# weights = [0.20716, 0.39316]

# models = [bayes_100, recency_ranks]
# weights = [0.34218, 0.39316]



reranking_coeff = 7
fusion_dic = fusiontools.reciprocal_rerank(models, reranking_coeff,  weights=weights)
# print(fusion_dic)

            
                

In [38]:
postprocess.write_results_ranked(fusion_dic, path_to_results, 'fusion-weighted-reranking-7-bayes-recency-tfidf-knn.csv')